<a href="https://colab.research.google.com/github/lemmingy/My_pipelines/blob/main/Sensor_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

センサーデータの変換

In [ ]:
#Tsfreshによる

In [ ]:
#センサーデータの変換